# FAT - Projet Vélib <a name="top"></a>


# Table of contents
1. [Simulation](#introduction)
    1. [Questions 5 & 6](#question5)
    2. [Question 7](#question7)
2. [Calcul théorique](#theorie)

### 1. Simulation <a name="introduction"></a> [$\uparrow$](#top)

Dans l'ensemble de cette section les matrices M de taille $NxN$ sont définies comme suit :
* sur la diagonale, les valeurs associées aux stations ($M[i][i]$ correspond à la station $i$)
* ailleurs, les valeurs associées aux transitions ($M[i][j]$ correspond à la transition $t_{ij}$, de $i$ vers $j$)

In [1]:
import random
import copy
import math

In [2]:
# Paramètres

N = 5

horizon = 150 * 60 * 60

tau = [[0,3,5,7,7],
      [2,0,2,5,5],
      [4,2,0,3,3],
      [8,6,4,0,2],
      [7,7,5,2,0]]

lambd_i = [2.8,3.7,5.5,3.5,4.6]

lambd = []
for i in range(N):
    lambd.append([])
    for j in range(N):
        if i==j:
            lambd[i].append(lambd_i[i] / 3600)
        else:
            lambd[i].append(1/(60*tau[i][j]))

p = [[0,0.22,0.32,0.2,0.26],
      [0.17,0,0.34,0.21,0.28],
      [0.19,0.26,0,0.24,0.31],
      [0.17,0.22,0.33,0,0.28],
      [0.18,0.24,0.35,0.23,0]]


In [3]:
# Lance une simulation avec les conditions initiales définies dans la fonction init() sur une période de temps "horizon"
def simulation(n_init):
    n = copy.deepcopy(n_init)
    temps = 0
    temps_vide = [0 for _ in range(N)]
    temps_vide2 = [[0 for _ in range(N)] for _ in range(N)]
    while temps < horizon:
        sum_q = 0
        for i in range(N):
            for j in range(N):
                if i == j:
                    if n[i][i] > 0:
                        sum_q += lambd[i][i]
                else:
                    sum_q += lambd[i][j] * n[i][j]
        tirage = random.expovariate(sum_q)
        temps += tirage
        # Calcul des temps vides
#        for i in range(N):
#            if(n[i][i] == 0):
#                temps_vide[i] += tirage
        for i in range(N):
            for j in range(N):
                if(n[i][j] == 0):
                    temps_vide2[i][j] += tirage

        #
        
        
        rand = random.random() * sum_q
        s = 0
        i = 0
        j = 0
        while 1 :
            if i == j:
                if n[i][i] > 0:
                    s += lambd[i][i]
            else:
                s += lambd[i][j] * n[i][j]
            if s >= rand :
                break
            if i == N-1:
                i = 0
                j += 1
            else:
                i += 1

        if i == j: # Départ de i
            s -= lambd[i][i]
            # On détermine la station destination
            j = 0
            while 1 :
                s += lambd[i][i] * p[i][j]
                if s >= rand:
                    break
                j += 1
            # On part de i vers j
            n[i][i] -= 1
            n[i][j] += 1
        else: # Arrivée en j à partir de i
            n[j][j] += 1
            n[i][j] -= 1
    return temps_vide2, n

In [4]:
# Lance K simulation avec les conditions initiales n_init
# Retourne :
# - empty_time (matrice (NxN)): le pourcentage moyen de temps où la colonie est vide durant une simulation
# - empty_end (matrice (NxN)) : le pourcentage de simulations à la fin desquelles la colonie est vide
# - epsilon (tableau (N)) : écart pour l'intervalle de confiance de la probabilité que la station soit vide 
def start_simulation(K, n_init):
    empty_time = [[0 for _ in range(N)] for _ in range(N)]
    empty_end = [[0 for _ in range(N)] for _ in range(N)]
    # Lancement des K simulations
    for _ in range(K):
        res, n_end = simulation(n_init)
        # Ajout des résultats partiels
        empty_time = [[empty_time[i][j] + res[i][j] for j in range(N)] for i in range(N)]
        empty_end = [[empty_end[i][j] + (1 if n_end[i][j] == 0 else 0) for j in range(N)] for i in range(N)] 
    # Calcul des moyennes
    empty_time = [[empty_time[i][j]/(K*horizon) for j in range(N)] for i in range(N)]
    empty_end = [[empty_end[i][j]/K for j in range(N)] for i in range(N)]
    
    # Calcul des variances pour chaque station
    var = [(empty_end[i][i] - empty_end[i][i]**2)*K/(K-1) for i in range(N)]
    # Calcul des epsilon
    epsilon = [1.96 * math.sqrt(var[i]) / math.sqrt(K) for i in range(N)]
    
    return empty_time, empty_end, epsilon

In [5]:
# Conditions initiales
n_init = [[20,1,0,0,0],
          [1,15,1,0,0],
          [0,1,17,1,0],
          [0,0,1,13,1],
          [0,0,0,1,18]]

K = 1000

empty_time, empty_end, epsilon = start_simulation(K,n_init)

#### A. Questions 5 & 6 <a name="question5"></a> [$\uparrow$](#top)

In [6]:
for i in range(N):
    print("La station", i, "est vide à", str(round(empty_end[i][i]*100,2)), "%.\
    Intervalle de confiance : ", round(100*(empty_end[i][i] - epsilon[i]),3), ";", round(100*(empty_end[i][i] + epsilon[i]),3))


La station 0 est vide à 1.0 %.    Intervalle de confiance :  0.383 ; 1.617
La station 1 est vide à 4.2 %.    Intervalle de confiance :  2.956 ; 5.444
La station 2 est vide à 15.9 %.    Intervalle de confiance :  13.632 ; 18.168
La station 3 est vide à 3.5 %.    Intervalle de confiance :  2.36 ; 4.64
La station 4 est vide à 12.8 %.    Intervalle de confiance :  10.728 ; 14.872


#### B. Question 7 <a name="question7"></a> [$\uparrow$](#top)

In [7]:
n_init = [[91,0,0,0,0],
          [0,0,0,0,0],
          [0,0,0,0,0],
          [0,0,0,0,0],
          [0,0,0,0,0]]

empty_time, empty_end, epsilon = start_simulation(K,n_init)

for i in range(N):
    print("La station", i, "est vide à", str(round(empty_end[i][i]*100,2)), "%.\
    Intervalle de confiance : ", round(100*(empty_end[i][i] - epsilon[i]),3), ";", round(100*(empty_end[i][i] + epsilon[i]),3))

La station 0 est vide à 0.1 %.    Intervalle de confiance :  -0.096 ; 0.296
La station 1 est vide à 7.4 %.    Intervalle de confiance :  5.777 ; 9.023
La station 2 est vide à 19.3 %.    Intervalle de confiance :  16.853 ; 21.747
La station 3 est vide à 6.5 %.    Intervalle de confiance :  4.971 ; 8.029
La station 4 est vide à 12.9 %.    Intervalle de confiance :  10.821 ; 14.979


In [8]:
n_init = [[0,0,0,45,0],
          [0,0,0,0,0],
          [0,1,0,0,0],
          [0,0,45,0,0],
          [0,0,0,0,0]]

empty_time, empty_end, epsilon = start_simulation(K,n_init)

for i in range(N):
    print("La station", i, "est vide à", str(round(empty_end[i][i]*100,2)), "%.\
    Intervalle de confiance : ", round(100*(empty_end[i][i] - epsilon[i]),3), ";", round(100*(empty_end[i][i] + epsilon[i]),3))

La station 0 est vide à 1.5 %.    Intervalle de confiance :  0.746 ; 2.254
La station 1 est vide à 3.7 %.    Intervalle de confiance :  2.529 ; 4.871
La station 2 est vide à 16.4 %.    Intervalle de confiance :  14.104 ; 18.696
La station 3 est vide à 1.1 %.    Intervalle de confiance :  0.453 ; 1.747
La station 4 est vide à 9.6 %.    Intervalle de confiance :  7.773 ; 11.427


### Calcul théorique <a name="theorie"></a> [$\uparrow$](#top)